In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# MARIA MAGDALENE NAMAGANDA
# 2019/HD07/24853U
# Kaggle assignment

## Importing some of the needed packages and libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split


> Now i need to get rid of some unecessary warnings 

In [ ]:
#To avoid unnecessary warnings, I will ignore them in the code below
import warnings
warnings.filterwarnings('ignore')

## Loading the datasets to be used
> There are two datasets, the train and test datasets 

In [ ]:
#Loading the datasets, 

Train = pd.read_csv("../input/amp-data-set/AMP_TrainSet.csv")
Test = pd.read_csv("../input/amp-data-set/Test.csv")



# Exploring my data
## Trying to know more about the data am dealing with

In [ ]:
#here, am trying to find the type of data
type(Train)
type(Test)
Train.dtypes
#Test.dtypes

In [ ]:
# checking the dimensions of the data
# this retuns the number of rows and columns in the data

Train.shape, Test.shape

#this helps to know how big the data is in terms of rows and columns.
#it also informs one of which data is labeled

## Data description
>for now, I will focus more on the train dataset because its what I will use to train the model 

In [ ]:
#getting a description of the data

Train.describe()

#description gives a summary of the data.

In [ ]:
#looking at the first 5 entries of my data
Train.head()

> *From the Train dataset, i see there is an extra class column which will be my validation set*

## Class Distribution

In [ ]:
Train.groupby('CLASS').size().plot(kind='bar')
pyplot.show()

> From the above barplot, I can see that the classes are evenly distributed so no need to use smote

> I would like to know how many instaces i have for each class

In [ ]:
#getting the number of instances in each class
Train.groupby('CLASS').size()

# Data Visualisation

## Looking at the correlation of the data

In [ ]:
#first ill checkfor the pairwise correlation of the attributes.
Train.corr(method='pearson')

In [ ]:
#Reviewing inter-correlation of attributes using heatmap
#graphical representation
plt.figure(figsize=(6,6))
sns.heatmap(Train.corr(method='pearson'))

In [ ]:
#Ill also check the correlation in regards to the 'CLASS' attribute
Train.corr(method='pearson')['CLASS']

In [ ]:
Train['CLASS'].value_counts

## Skewnwess of the data
> knowing data skewness allows one to perform data preparation and improve a model

In [ ]:
Train.skew().plot(kind='bar')

# Comparing Models

In [ ]:
#comparing different models to from which ill choose.


# load dataset

array = Train.values

#split the dataset 
X = array[:,0:11]  #X = Train.drop(columns=['CLASS'])
Y = array[:,11]   #Y = Train['CLASS']

# prepare models and add them to a list
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# evaluate each model in turn
results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=40, random_state=10)
    cv_results = cross_val_score(model, X, Y, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = (name, cv_results.mean(), cv_results.std())
    print(msg)

# boxplot algorithm comparison
fig = pyplot.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

# Feature selection 
## Using recursive feature elimination

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

array = Train.values
X = array[:,0:11]
Y = array[:,11]
# feature extraction
model = LogisticRegression()
rfe = RFE(model, 4)
fit = rfe.fit(X, Y)
print("Num Features: ", fit.n_features_)
print("Selected Features:", fit.support_)
print("Feature Ranking: ", fit.ranking_)

In [ ]:
#Calling out the column names so I can know which features am going to drop from RFE
Train.columns

## Using Feature importance

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

array = Train.values
X = array[:,0:11]
Y = array[:,11]
# feature extraction
model = ExtraTreesClassifier()
model.fit(X, Y)
print(model.feature_importances_)

## If am to use 4 features and drop the rest

In [ ]:
# I will call the new Train data with selected features New_Train.
Train
New_Train4 = Train.drop(['FULL_Charge', 'FULL_AcidicMolPerc', 'FULL_DAYM780201', 'FULL_GEOR030101', 'AS_MeanAmphiMoment', 'AS_DAYM780201', 'AS_FUKS010112'], axis =1)

In [ ]:
#dropping the same features in the test dataset
New_Test4 = Test.drop(['FULL_Charge', 'FULL_AcidicMolPerc', 'FULL_DAYM780201', 'FULL_GEOR030101', 'AS_MeanAmphiMoment', 'AS_DAYM780201', 'AS_FUKS010112'], axis =1)

In [ ]:
#viewing tselected features
New_Train4.columns

# Rescaling data

In [ ]:
from numpy import set_printoptions
from sklearn.preprocessing import MinMaxScaler
array = New_Train4.values

#seperating data into onput and output options
X = array[:,0:4]
Y = array[:,4]
scaler = MinMaxScaler(feature_range = (0,1))
rescaledX = scaler.fit_transform(X)

#summarising transformed data
set_printoptions(precision = 3)
print(rescaledX[0:4,:])
    

# Standardising data

In [ ]:
from sklearn.preprocessing import StandardScaler


In [ ]:
#now splitting data
#array = New_Train.values
X = New_Train4.values[:,0:4]
Y = New_Train4.values[:,4]
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# also train and test the model on Matthews correlation coefficient.
from sklearn.metrics import matthews_corrcoef

GS = GaussianNB()
GS.fit(X_Train,Y_Train)
pred = GS.predict(X_Test)

print("The result is: ",np.round(matthews_corrcoef(Y_Test,pred) *100,2)," Mathew's Coef")

## Using four features gives me a very low MCC and low overall score.
## I'll therefore consider using 8 features and see what score  get

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

array = Train.values
X = array[:,0:11]
Y = array[:,11]
# feature extraction
model = LogisticRegression()
rfe = RFE(model, 8)
fit = rfe.fit(X, Y)
print("Num Features: ", fit.n_features_)
print("Selected Features:", fit.support_)
print("Feature Ranking: ", fit.ranking_)

In [ ]:
# I will call the new Train data with selected features New_Train.
Train
New_Train8 = Train.drop(['FULL_AcidicMolPerc', 'FULL_DAYM780201', 'AS_DAYM780201'], axis =1)

In [ ]:
#dropping the same features in the test dataset
New_Test8 = Test.drop(['FULL_AcidicMolPerc', 'FULL_DAYM780201', 'AS_DAYM780201'], axis =1)

In [ ]:
#now splitting data
#array = New_Train.values


X = New_Train8.values[:,0:8]
Y = New_Train8.values[:,8]
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
#now creating a model and training it on 8 features
model = LogisticRegression(solver='liblinear', C=0.05, multi_class='ovr', random_state=30)
model.fit(X_Train, Y_Train)

In [ ]:
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,
random_state=42)

In [ ]:
from sklearn.metrics import matthews_corrcoef

nv = GaussianNB()
nv.fit(X_Train,Y_Train)
pred = nv.predict(X_Test)

print("The result is: ",np.round(matthews_corrcoef(Y_Test,pred) *100,2)," Mathew's Coef")

# Logistic Regression using 8 features

In [ ]:
array = New_Train8.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.30
seed = 7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,
random_state=seed)
model = LogisticRegression()
model.fit(X_train, Y_train)
result = model.score(X_test, Y_test)
print("Accuracy: ",  (result*100.0))

In [ ]:
Test.head()

In [ ]:
#we now need to predict on the test dataset
#md = pd.DataFrame((New_Test.index,nv.predict(New_Test))).T
#md = md.rename(columns={0:"Index",1:"CLASS"}).set_index('Index')
#md.to_csv('maria.csv')

In [ ]:
#we now need to predict on the test dataset
md = pd.DataFrame((New_Test8.index,nv.predict(New_Test8))).T
md = md.rename(columns={0:"Index",1:"CLASS"}).set_index('Index')
md.to_csv('maria1.csv')

In [ ]:
import os
os.listdir()
!ls ../../kaggle/working/

# Using Naive Bayes and all the features

In [ ]:
array = Train.values
X = array[:,0:11]
Y = array[:,11]
test_size = 0.35
seed = 3
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,
random_state=seed)
model = GaussianNB()
model.fit(X_train, Y_train)
result = model.score(X_test, Y_test)
prediction=model.predict(X_test)
matrix=matthews_corrcoef(Y_test,prediction)
print("Accuracy: ",  (result*100.0))
print(matrix)

In [ ]:
array = Train.values
X = array[:,0:11]
Y = array[:,11]
kfold = KFold(n_splits=10, random_state=7)
model = GaussianNB()
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())


model.fit(X,Y)
output = model.predict(Test.values)

from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(model.predict(X),Y)
print('MCC:',mcc)
                       
maria2_bayes = pd.DataFrame(output)
maria2_bayes.columns = ['CLASS']
maria2_bayes.index.name = "Index"
maria2_bayes['CLASS']=maria2_bayes['CLASS'].map({0.0:False, 1.0:True})
maria2_bayes.to_csv("maria2_bayes.csv")


print(maria2_bayes['CLASS'].unique())
print('False: ',maria2_bayes.groupby('CLASS').size()[0].sum())
print('True: ',maria2_bayes.groupby('CLASS').size()[1].sum())